In [1]:
import gzip
import pickle
import random
import numpy as np

In [3]:
def load_data():
  """Returns MNIST data as a tuple with training data, validation data, test data"""
  with gzip.open('mnist.pkl.gz', 'rb') as file:
    training_data, validation_data, test_data = pickle.load(file, encoding='latin1')
  return (training_data, validation_data, test_data)

def load_data_wrapper():
  tr_d, va_d, te_d = load_data()
  training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
  training_results = [vectorized_result(y) for y in tr_d[1]]
  training_data = list(zip(training_inputs, training_results))
  validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
  validation_data = list(zip(validation_inputs, va_d[1]))
  test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
  test_data = list(zip(test_inputs, te_d[1]))
  return (training_data, validation_data, test_data)

def vectorized_result(j):
  """Return a 10-dimensional unit vector with a 1.0 in the jth position and zeroes elsewhere.
  This is used to convert a digit (0...9) into a corresponding desired output from the neural network."""
  e = np.zeros((10, 1))
  e[j] = 1.0
  return e

In [4]:
# Activation function
def sigmoid(z):
  return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
  return sigmoid(z) * (1 - sigmoid(z))

# Cost functions
class QuadraticCost:
  @staticmethod
  def fn(a, y):
    return 0.5 * np.linalg.norm(a - y) ** 2

  @staticmethod
  def delta(z, a, y):
    return (a - y) * sigmoid_prime(z)

class CrossEntropyCost:
  @staticmethod
  def fn(a, y):
    return np.sum(np.nan_to_num(-y*np.log(a) - (1-y)*np.log(1-a)))

  @staticmethod
  def delta(z, a, y):
    return (a - y)

In [18]:
class Network:
  """The list ``sizes`` contains the number of neurons in the respective layers of the network."""
  def __init__(self, sizes, cost=CrossEntropyCost):
    self.num_layers = len(sizes)
    self.sizes = sizes
    self.default_weight_initializer()
    self.cost = cost

  def default_weight_initializer(self):
    self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
    self.weights = [np.random.randn(y, x) / np.sqrt(x) for x, y in zip(self.sizes[:-1], self.sizes[1:])]

  def feedforward(self, a):
    for b, w in zip(self.biases, self.weights):
      a = sigmoid(np.dot(w, a) + b)
    return a

  def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None, monitor_evaluation_accuracy=False):
    n = len(training_data)
    if test_data:
      n_test = len(test_data)
    for j in range(epochs):
      random.shuffle(training_data)
      mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
      for mini_batch in mini_batches:
        self.update_mini_batch(mini_batch, eta)
      if test_data:
        print(f"Epoch {j+1}, Accuracy: {self.evaluate(test_data)} / {len(test_data)}")
      else:
        print()

  def update_mini_batch(self, mini_batch, eta):
    """Update the network's weights and biases by applying gradient descent using
    backpropagation to a single mini batch. The "mini_batch" is a list of tuples "(x, y)",
    and "eta" is the learning rate."""
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    for x, y in mini_batch:
      delta_nabla_b, delta_nabla_w = self.backprop(x, y)
      nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
      nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.weights = [w - (eta / len(mini_batch)) * nw
                      for w, nw in zip(self.weights, nabla_w)]
    self.biases = [b - (eta / len(mini_batch)) * nb
                      for b, nb in zip(self.biases, nabla_b)]

  def backprop(self, x, y):
    """Return a tuple ``(nabla_b, nabla_w)`` representing the gradient for the cost function C_x.
    ``nabla_b`` and ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
    to ``self.biases`` and ``self.weights``."""
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]

    # forward pass
    activation = x
    activations = [x]  # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    for b, w in zip(self.biases, self.weights):
      z = np.dot(w, activation) + b
      zs.append(z)
      activation = sigmoid(z)
      activations.append(activation)

    # backward pass
    delta = self.cost.delta(zs[-1], activations[-1], y)
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].T)
    for l in range(2, self.num_layers):
      z = zs[-l]
      sp = sigmoid_prime(z)
      delta = np.dot(self.weights[-l + 1].T, delta) * sp
      nabla_b[-l] = delta
      nabla_w[-l] = np.dot(delta, activations[-l - 1].T)
    return (nabla_b, nabla_w)

  def evaluate(self, test_data):
    test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

  def cost_derivative(self, output_activations, y):
    return output_activations - y

training_data, validation_data, test_data = load_data_wrapper()

In [19]:
net = Network([784, 30, 10], cost=CrossEntropyCost)
net.SGD(training_data, epochs=30, mini_batch_size=10, eta=0.5, test_data=test_data)

Epoch 1, Accuracy: 9387 / 10000
Epoch 2, Accuracy: 9346 / 10000
Epoch 3, Accuracy: 9462 / 10000
Epoch 4, Accuracy: 9526 / 10000
Epoch 5, Accuracy: 9531 / 10000
Epoch 6, Accuracy: 9554 / 10000
Epoch 7, Accuracy: 9565 / 10000
Epoch 8, Accuracy: 9546 / 10000
Epoch 9, Accuracy: 9571 / 10000
Epoch 10, Accuracy: 9567 / 10000
Epoch 11, Accuracy: 9579 / 10000
Epoch 12, Accuracy: 9568 / 10000
Epoch 13, Accuracy: 9567 / 10000
Epoch 14, Accuracy: 9580 / 10000
Epoch 15, Accuracy: 9567 / 10000
Epoch 16, Accuracy: 9588 / 10000
Epoch 17, Accuracy: 9564 / 10000
Epoch 18, Accuracy: 9579 / 10000
Epoch 19, Accuracy: 9592 / 10000
Epoch 20, Accuracy: 9584 / 10000
Epoch 21, Accuracy: 9562 / 10000
Epoch 22, Accuracy: 9583 / 10000
Epoch 23, Accuracy: 9566 / 10000
Epoch 24, Accuracy: 9597 / 10000
Epoch 25, Accuracy: 9571 / 10000
Epoch 26, Accuracy: 9579 / 10000
Epoch 27, Accuracy: 9575 / 10000
Epoch 28, Accuracy: 9589 / 10000
Epoch 29, Accuracy: 9589 / 10000
Epoch 30, Accuracy: 9568 / 10000


In [20]:
net = Network([784, 30, 10], cost=QuadraticCost)
net.SGD(training_data, epochs=30, mini_batch_size=10, eta=0.5, test_data=test_data)

Epoch 1, Accuracy: 9185 / 10000
Epoch 2, Accuracy: 9349 / 10000
Epoch 3, Accuracy: 9412 / 10000
Epoch 4, Accuracy: 9447 / 10000
Epoch 5, Accuracy: 9478 / 10000
Epoch 6, Accuracy: 9488 / 10000
Epoch 7, Accuracy: 9519 / 10000
Epoch 8, Accuracy: 9543 / 10000
Epoch 9, Accuracy: 9546 / 10000
Epoch 10, Accuracy: 9547 / 10000
Epoch 11, Accuracy: 9555 / 10000
Epoch 12, Accuracy: 9570 / 10000
Epoch 13, Accuracy: 9579 / 10000
Epoch 14, Accuracy: 9573 / 10000
Epoch 15, Accuracy: 9575 / 10000
Epoch 16, Accuracy: 9584 / 10000
Epoch 17, Accuracy: 9587 / 10000
Epoch 18, Accuracy: 9585 / 10000
Epoch 19, Accuracy: 9566 / 10000
Epoch 20, Accuracy: 9578 / 10000
Epoch 21, Accuracy: 9589 / 10000
Epoch 22, Accuracy: 9600 / 10000
Epoch 23, Accuracy: 9604 / 10000
Epoch 24, Accuracy: 9590 / 10000
Epoch 25, Accuracy: 9596 / 10000
Epoch 26, Accuracy: 9597 / 10000
Epoch 27, Accuracy: 9600 / 10000
Epoch 28, Accuracy: 9591 / 10000
Epoch 29, Accuracy: 9600 / 10000
Epoch 30, Accuracy: 9601 / 10000
